In [1]:
import numpy as np
from PIL import Image
from pathlib import Path
from scipy import stats
import matplotlib.pyplot as plt
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
import cv2


In [2]:
points = set()

In [3]:
for subj_id in range(23):
    output_data = Path(f'output-data/S{subj_id:03d}')
    mel_paths = sorted(output_data.glob('**/out-mel/*.txt'))
    hg_paths = sorted(output_data.glob('**/out-hem/*.txt'))
    epth_paths = sorted(output_data.glob('**/out-epth/*.txt'))

    
    mels =[np.loadtxt(f, delimiter=',') for f in mel_paths]
    hgs = [np.loadtxt(f, delimiter=',') for f in hg_paths]
    epths = [np.loadtxt(f, delimiter=',') for f in epth_paths]

    for i in range(len(mel_paths)):
        patch_mels = mels[i]
        patch_hgs = hgs[i]
        patch_epths = epths[i]

        h, w = patch_mels.shape
        
        for row in range(h):
            for col in range(w):
                mel_p = patch_mels[row,col]
                hg_p = patch_hgs[row,col]
                epth_p = patch_epths[row,col] * 0.001

                points.add((mel_p,hg_p,epth_p))
    
    print(f"S{subj_id:03d}")


S000
S001
S002
S003
S004
S005
S006
S007
S008
S009
S010
S011
S012
S013
S014
S015
S016
S017
S018
S019
S020
S021
S022


In [4]:
points = list(points)

In [5]:
template_file = ""
mel_string = "MEL"
hg_string = "HG"
epth_string = "EPTH"

In [7]:
with open("template-pixel.pbrt", "r") as f:
    template_file = f.read()

assert template_file != ""

In [8]:
for i in range(len(points)):
        mel = points[i][0]
        hg = points[i][1]
        epth = points[i][2]

        output = template_file.replace(mel_string, f'{mel}')
        output = output.replace(hg_string, f'{hg}')
        output = output.replace(epth_string, f'{epth}e6')

        with open(f"pbrt-inputs-per-point/{mel}mel-{hg}hg-{epth}epth.pbrt", "w") as f:
            f.write(output)

# ---------------------------------------------------------------------------------------

In [2]:
for subj_id in range(23):
    output_data = Path(f'output-data/S{subj_id:03d}')
    mel_paths = sorted(output_data.glob('**/out-mel/*.txt'))
    hg_paths = sorted(output_data.glob('**/out-hem/*.txt'))
    epth_paths = sorted(output_data.glob('**/out-epth/*.txt'))

    
    mels =[np.loadtxt(f, delimiter=',') for f in mel_paths]
    hgs = [np.loadtxt(f, delimiter=',') for f in hg_paths]
    epths = [np.loadtxt(f, delimiter=',') for f in epth_paths]

    Path(f"interpolated-patches-per-point/S{subj_id:03d}").mkdir(parents=True, exist_ok=True)

    for i in range(len(mel_paths)):
        patch_mels = mels[i]
        patch_hgs = hgs[i]
        patch_epths = epths[i]

        h, w = patch_mels.shape
        interpolated = np.zeros((h,w,3), dtype='float32')

        for row in range(h):
            for col in range(w):
                mel_p = patch_mels[row,col]
                hg_p = patch_hgs[row,col]
                epth_p = patch_epths[row,col] * 0.001

                
                interpolated[row,col,:] = cv2.imread(f"pbrt-outputs-per-point/{mel_p}mel-{hg_p}hg-{epth_p}epth.exr", cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
                
        cv2.imwrite(f"interpolated-patches-per-point/S{subj_id:03d}/{i:03d}.exr", interpolated)